### Variabilitätsscore für die zehn gewaehlten Orte berechnen

In [1]:
# Daten einlesen
wetterdaten <- readRDS("datasets/rds/wetterdaten.rds")

In [2]:
# Dataframe für die Standardabweichungen erzeugen
n <- length(wetterdaten)
standardabweichungen_df <- data.frame(
  id = c("00856", "00891", "01346", "02290", "02712", "03631", "03730", "03987", "04271", "04625"),
  name = c("Chieming", "Cuxhaven", "Feldberg/Schwarzwald", "Hohenpeißenberg", "Konstanz", "Norderney", "Oberstdorf", "Potsdam", "Rostock-Warnemünde", "Schwerin"),
  STDA_RSK = numeric(n),
  STDA_SDK = numeric(n),
  STDA_TMK = numeric(n)
)

# Standardabweichung für jeden Ort in der Liste berechnen
for (i in seq_along(wetterdaten)) {
  standardabweichungen_df$STDA_RSK[i] <- sqrt(var(wetterdaten[[i]]$RSK, na.rm = TRUE))
  standardabweichungen_df$STDA_SDK[i] <- sqrt(var(wetterdaten[[i]]$SDK, na.rm = TRUE))
  standardabweichungen_df$STDA_TMK[i] <- sqrt(var(wetterdaten[[i]]$TMK, na.rm = TRUE))
}

# Ausgabe des Dataframes
print(standardabweichungen_df)

      id                 name STDA_RSK STDA_SDK STDA_TMK
1  00856             Chieming 6.250016 4.659858 7.376363
2  00891             Cuxhaven 4.380597 4.442633 6.133311
3  01346 Feldberg/Schwarzwald 8.938837 4.478617 7.095335
4  02290      Hohenpeißenberg 7.079220 4.613091 7.480038
5  02712             Konstanz 5.608673 4.695837 7.372305
6  03631            Norderney 4.052408 4.558483 5.810113
7  03730           Oberstdorf 8.792411 4.049344 7.539743
8  03987              Potsdam 4.122642 4.607992 7.515048
9  04271   Rostock-Warnemünde 3.793048 4.807175 6.533197
10 04625             Schwerin 3.792455 4.448527 6.976120


In [3]:
# Gewichtungen festlegen
gewicht_rsk <- 0.2
gewicht_sdk <- 0.2
gewicht_tmk <- 0.6

# Variabilitätsscore als gewichtete Summe der Standardabweichungen berechnen
standardabweichungen_df$Variabilitaetsscore <- (standardabweichungen_df$STDA_RSK * gewicht_rsk) +
                                               (standardabweichungen_df$STDA_SDK * gewicht_sdk) +
                                               (standardabweichungen_df$STDA_TMK * gewicht_tmk)

print(standardabweichungen_df[order(standardabweichungen_df$Variabilitaetsscore), ])

      id                 name STDA_RSK STDA_SDK STDA_TMK Variabilitaetsscore
6  03631            Norderney 4.052408 4.558483 5.810113            5.208246
2  00891             Cuxhaven 4.380597 4.442633 6.133311            5.444633
9  04271   Rostock-Warnemünde 3.793048 4.807175 6.533197            5.639963
10 04625             Schwerin 3.792455 4.448527 6.976120            5.833869
8  03987              Potsdam 4.122642 4.607992 7.515048            6.255155
5  02712             Konstanz 5.608673 4.695837 7.372305            6.484285
1  00856             Chieming 6.250016 4.659858 7.376363            6.607793
4  02290      Hohenpeißenberg 7.079220 4.613091 7.480038            6.826485
3  01346 Feldberg/Schwarzwald 8.938837 4.478617 7.095335            6.940692
7  03730           Oberstdorf 8.792411 4.049344 7.539743            7.092197


In [4]:
# Ranking dataframe erstellen
rank_df <- data.frame(Name = standardabweichungen_df$name, Variabilitaetsscore = standardabweichungen_df$Variabilitaetsscore)

# Sortiere den DataFrame für eine bessere Visualisierung
rank_df <- rank_df[order(rank_df$Variabilitaetsscore, decreasing = TRUE), ]

# Erstelle ein Balkendiagramm
if (!require(ggplot2, quietly = TRUE)) {
  install.packages("ggplot2")
}

png_name <- paste("plots/rank/", "rank_all.png", sep = "")
png(png_name, width=800, height=400)

ggplot(rank_df, aes(x = reorder(Name, Variabilitaetsscore), y = Variabilitaetsscore, fill = Name)) +
  geom_bar(stat = "identity") +
  theme_minimal(base_size = 16) +
  labs(x = "", y = "Variabilitätsscore") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

dev.off()

png 
  2

### Funktion für den Vergleich des Variabilitaetsscores von zwei Orten 

In [9]:
Orte_vergleichen <- function(df1, df2, w1, w2, w3) {

  # Überprüfen, ob die Dataframes die benötigten Spalten enthalten
  erforderlicheSpalten <- c("STATIONS_ID", "RSK", "SDK", "TMK")
  if (!all(erforderlicheSpalten %in% names(df1)) | !all(erforderlicheSpalten %in% names(df2))) {
    stop("Einer der Dataframes enthält nicht alle erforderlichen Spalten.")
  }
  # Ergebnis dataframe erstellen
  standardabweichungen_df <- data.frame(
  STATIONS_ID = c(df1$STATIONS_ID[1], df2$STATIONS_ID[1]),
  STDA_RSK = numeric(2),
  STDA_SDK = numeric(2),
  STDA_TMK = numeric(2)
  )

  # Berechnung der Standardabweichungen für beide Dataframes
  standardabweichungen_df$STDA_RSK[1] <- sqrt(var(df1$RSK))
  standardabweichungen_df$STDA_SDK[1] <- sqrt(var(df1$SDK))
  standardabweichungen_df$STDA_TMK[1] <- sqrt(var(df1$TMK))
  standardabweichungen_df$STDA_RSK[2] <- sqrt(var(df2$RSK))
  standardabweichungen_df$STDA_SDK[2] <- sqrt(var(df2$SDK))
  standardabweichungen_df$STDA_TMK[2] <- sqrt(var(df2$TMK))

  # Gewichtete Summe berechnen
  standardabweichungen_df$Variabilitaetsscore <- (standardabweichungen_df$STDA_RSK * w1) +
                                                 (standardabweichungen_df$STDA_SDK * w2) +
                                                 (standardabweichungen_df$STDA_TMK * w3)

  # Ergebnis plotten
  png_name <- paste("plots/rank/", "rank", df1$STATIONS_ID[1], "_", df2$STATIONS_ID[1], ".png", sep = "")
  png(png_name, width=300, height=400)
  
  plot <- ggplot(
    standardabweichungen_df, 
    aes(x=factor(STATIONS_ID), y=Variabilitaetsscore, fill=factor(STATIONS_ID))) + 
    geom_bar(stat="identity", width = 0.7) +
    labs(x="Stations ID", y="Variabilitätsscore", title="") +
    theme_minimal(base_size = 16) + 
    guides(fill = FALSE)
  print(plot)
  dev.off()

  # Ergebnis zurückgeben
  file <- paste("outputs/", df1$STATIONS_ID[1], "_", df2$STATIONS_ID[1], ".csv", sep = "")
  write.csv2(standardabweichungen_df, file, row.names = FALSE,)

  return(standardabweichungen_df)
}

# Testen der Funktion
Orte_vergleichen(wetterdaten[[1]], wetterdaten[[2]], 0.2, 0.2, 0.6)


STATIONS_ID,STDA_RSK,STDA_SDK,STDA_TMK,Variabilitaetsscore
<int>,<dbl>,<dbl>,<dbl>,<dbl>
856,6.250016,4.659858,7.376363,6.607793
891,4.380597,4.442633,6.133311,5.444633
